# 引入相依性

In [ ]:
import torch.nn as nn
import torch
import os
import sys
import numpy as np
from PIL import Image as images
from torchvision import transforms
import torch.optim as optim

In [ ]:
device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
def 確認GPU():
    device = (torch.device('cuda') if torch.cuda.is_available()
    else torch.device('cpu'))
    print(f"Training on device {device}.")
確認GPU()

# 讀取資料

In [ ]:
group = os.listdir("test")
toTensor = transforms.ToTensor()
resize = transforms.Resize([256,256])
testTensorArray=[]
trainTensorArray=[]
index = 0
for label in group:
    testlabels = os.listdir("test/"+label)
    trainlabels = os.listdir("train/"+label)
    for j in testlabels:
        testReadImage = resize(images.open("test/%s/%s" %(label,j)))
        testTensorImg = toTensor(testReadImage)
        if testTensorImg.shape[0] != 3:
            testTensorImg=testTensorImg.expand([3,-1,-1])
        testTensorArray.append((testTensorImg,index))
    for j in trainlabels:
        trainReadImage = resize(images.open("train/%s/%s" %(label,j)))
        trainTensorImg = toTensor(trainReadImage)
        if trainTensorImg.shape[0] != 3:
            trainTensorImg=trainTensorImg.expand([3,-1,-1])
        trainTensorArray.append((trainTensorImg,index))
    index = index+1

# 建model

In [ ]:
model = nn.Sequential(
    nn.Linear(196608,8192),
    nn.Tanh(),
    nn.Linear(8192,1024),
    nn.Tanh(),
    nn.Linear(1024,10),
    nn.ReLU(),
    nn.Softmax(dim=1)
)

# 定義參數

In [ ]:
學習率 = 0.01
損失函數 = nn.CrossEntropyLoss()
優化器 = optim.SGD(model.parameters(),lr=學習率)
epoches = 8192

# Loading test set

In [ ]:
訓練集 = torch.utils.data.DataLoader(trainTensorArray, batch_size=1, shuffle=True)
測試集 = torch.utils.data.DataLoader(testTensorArray, batch_size=1, shuffle=False)

# 訓練

In [ ]:
model = model.to(device)
for epoch in range(epoches):
    for imgs, labels in 訓練集:
        torch.cuda.empty_cache()
        batch_size = imgs.shape[0]
        outputs = model(imgs.to(device).view(batch_size, -1))
        loss = 損失函數(outputs, labels.to(device))
        優化器.zero_grad()
        loss.backward()
        優化器.step()
    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

# 驗證

In [ ]:
correct = 0
total = 0
model = model.to(device)
with torch.no_grad():
    for imgs, labels in 測試集:
        torch.cuda.empty_cache()
        batch_size = imgs.shape[0]
        outputs = model(imgs.to(device).view(batch_size, -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels.to(device)).sum())
print("Accuracy: %f", correct / total)
model=model.cpu()
torch.cuda.empty_cache()